# TextGAN

In [1]:
#Solo si se está usando Google Colab
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
#Solo si se está usando Google Colab
#Solo ejecutar una vez, es para clonar el repo en drive (para tener las salidas generadas guardadas en el drive en caso de que se apague la maquina).
!cd /content/gdrive/MyDrive && git clone -b Thesis https://github.com/YuiZh0u/TextGAN-PyTorch.git

## Instalación de requisitos

In [2]:
!cd /content/gdrive/MyDrive/TextGAN-PyTorch && pip install -r requirements.txt

     |████████████████████████████████| 1.5 MB 12.0 MB/s 
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
!apt-get install libboost-all-dev

In [ ]:
!git clone https://github.com/kpu/kenlm.git

In [ ]:
!cd kenlm && mkdir -p build && cd build && cmake .. && make -j 4

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# !cp /content/gdrive/MyDrive/word2vec/covid_tweets_equal16.txt /content/gdrive/MyDrive/TextGAN-PyTorch/dataset/testdata/

In [ ]:
!cd /content/gdrive/MyDrive/TextGAN-PyTorch/run && python3 run_seqgan.py 3 0

job_id: 3, gpu_id: 0
total tokens:  32666
> training arguments:
>>> if_test: 0
>>> run_model: seqgan
>>> k_label: 2
>>> dataset: covid_tweets
>>> model_type: vanilla
>>> loss_type: rsgan
>>> mu_type: ragan
>>> eval_type: Ra
>>> d_type: Ra
>>> if_real_data: 1
>>> cuda: 1
>>> device: 0
>>> devices: 0
>>> shuffle: 0
>>> gen_init: normal
>>> dis_init: uniform
>>> n_parent: 1
>>> eval_b_num: 8
>>> lambda_fq: 1.0
>>> lambda_fd: 0.0
>>> d_out_mean: True
>>> freeze_dis: False
>>> freeze_clas: False
>>> use_all_real_fake: False
>>> use_population: False
>>> samples_num: 10000
>>> vocab_size: 32666
>>> mle_epoch: 120
>>> clas_pre_epoch: 10
>>> adv_epoch: 200
>>> inter_epoch: 15
>>> batch_size: 64
>>> max_seq_len: 66
>>> start_letter: 1
>>> padding_idx: 0
>>> gen_lr: 0.01
>>> gen_adv_lr: 0.0001
>>> dis_lr: 0.0001
>>> clip_norm: 5.0
>>> pre_log_step: 10
>>> adv_log_step: 1
>>> train_data: dataset/covid_tweets.txt
>>> test_data: dataset/testdata/covid_tweets_test.txt
>>> temp_adpt: exp
>>> evo_temp